In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/bike-sharing-dataset/hour.csv
/kaggle/input/bike-sharing-dataset/Readme.txt
/kaggle/input/bike-sharing-dataset/day.csv
/kaggle/input/bike-sharing-dataset/bike-sharing-dataset/hour.csv
/kaggle/input/bike-sharing-dataset/bike-sharing-dataset/Readme.txt
/kaggle/input/bike-sharing-dataset/bike-sharing-dataset/day.csv


In [2]:
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [3]:
df = pd.read_csv("../input/bike-sharing-dataset/day.csv")

In [4]:
df.head()

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,6,0,2,0.344167,0.363625,0.805833,0.160446,331,654,985
1,2,2011-01-02,1,0,1,0,0,0,2,0.363478,0.353739,0.696087,0.248539,131,670,801
2,3,2011-01-03,1,0,1,0,1,1,1,0.196364,0.189405,0.437273,0.248309,120,1229,1349
3,4,2011-01-04,1,0,1,0,2,1,1,0.200000,0.212122,0.590435,0.160296,108,1454,1562
4,5,2011-01-05,1,0,1,0,3,1,1,0.226957,0.229270,0.436957,0.186900,82,1518,1600


In [5]:
df.columns

Index(['instant', 'dteday', 'season', 'yr', 'mnth', 'holiday', 'weekday',
       'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed',
       'casual', 'registered', 'cnt'],
      dtype='object')

In [6]:
df.drop(['instant', 'dteday','casual','registered'], inplace=True ,axis=1)

**Question 4. a)**

In [7]:
correlations = df.corrwith(df['cnt'])
print(correlations)

season        0.406100
yr            0.566710
mnth          0.279977
holiday      -0.068348
weekday       0.067443
workingday    0.061156
weathersit   -0.297391
temp          0.627494
atemp         0.631066
hum          -0.100659
windspeed    -0.234545
cnt           1.000000
dtype: float64


Features with positive correlation are: 
* **season**
* **yr**: year
* **mnth**: mmonths
* **weekday**: days of the week
* **workingday**: whether the day is neither a weekend nor holiday
* **temp**: temperature in Celsius
* **atemp**: "feels like" temperature in Celsius

The feature with highest correlation is **'atemp'**.

**Question 4. b)**

Features with negative correlation are: 
* **holiday**: whether the day is considered a holiday
* **weathersit**
* **hum**: humidity
* **windspeed**

In [8]:
df.head()

,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,cnt
0,1,0,1,0,6,0,2,0.344167,0.363625,0.805833,0.160446,985
1,1,0,1,0,0,0,2,0.363478,0.353739,0.696087,0.248539,801
2,1,0,1,0,1,1,1,0.196364,0.189405,0.437273,0.248309,1349
3,1,0,1,0,2,1,1,0.200000,0.212122,0.590435,0.160296,1562
4,1,0,1,0,3,1,1,0.226957,0.229270,0.436957,0.186900,1600


**Question 4.c)**

In [9]:
import category_encoders as ce

In [10]:
encoder = ce.OneHotEncoder(cols=('season','yr','mnth','holiday','weekday','weathersit'), use_cat_names = True)

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
encoded = encoder.fit_transform(df)
new_df = pd.concat([df.drop(['season','yr','mnth','holiday','weekday','weathersit','cnt'],axis=1),encoded],axis=1)

In [13]:
new_df.head()

,workingday,temp,atemp,hum,windspeed,season_1.0,season_2.0,season_3.0,season_4.0,yr_0.0,...,weekday_5.0,workingday,weathersit_2.0,weathersit_1.0,weathersit_3.0,temp,atemp,hum,windspeed,cnt
0,0,0.344167,0.363625,0.805833,0.160446,1,0,0,0,1,...,0,0,1,0,0,0.344167,0.363625,0.805833,0.160446,985
1,0,0.363478,0.353739,0.696087,0.248539,1,0,0,0,1,...,0,0,1,0,0,0.363478,0.353739,0.696087,0.248539,801
2,1,0.196364,0.189405,0.437273,0.248309,1,0,0,0,1,...,0,1,0,1,0,0.196364,0.189405,0.437273,0.248309,1349
3,1,0.200000,0.212122,0.590435,0.160296,1,0,0,0,1,...,0,1,0,1,0,0.200000,0.212122,0.590435,0.160296,1562
4,1,0.226957,0.229270,0.436957,0.186900,1,0,0,0,1,...,0,1,0,1,0,0.226957,0.229270,0.436957,0.186900,1600


In [14]:
X = new_df.drop(['cnt','workingday'], axis = 1)
y = new_df['cnt']

**Workingday** is dropped because it is related with holiday 

In [15]:
y

0       985
1       801
2      1349
3      1562
4      1600
       ... 
726    2114
727    3095
728    1341
729    1796
730    2729
Name: cnt, Length: 731, dtype: int64

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [17]:
from sklearn.linear_model import LinearRegression

In [18]:
LR = LinearRegression()

In [19]:
LR.fit(X_train, y_train)

LinearRegression()

In [20]:
LR.coef_

array([-1.22890549e+16,  1.84050000e+03, -6.52083811e+02, -1.10960252e+03,
       -9.46236603e+02,  1.10402434e+02,  1.16859425e+02,  7.86333604e+02,
       -9.91436156e+02,  9.71052084e+02, -7.61286502e+01,  5.92396907e+01,
        4.90914051e+02,  8.37791449e+01,  3.26381701e+02,  1.47548247e+01,
       -5.47806058e+02, -2.31291306e+02,  5.58128836e+02,  1.83645124e+02,
       -4.26070831e+02, -3.40710852e+02,  2.85023358e+02, -2.90367108e+02,
        1.17002260e+02, -3.42486123e+02, -5.21637419e+01, -4.73031381e+01,
        6.56444483e+01,  6.77283970e+01,  1.36528345e+02,  2.31578465e+02,
        7.52826270e+02, -1.04573438e+03,  1.22890549e+16,  1.90175000e+03,
       -6.93265625e+02, -1.15386719e+03])

In [21]:
pred = LR.predict(X_train)

In [22]:
from sklearn.metrics import mean_squared_error,r2_score

**Question 4. c)**

The rmse and r^2 value for training dataset using Linear Regression and Random Forest are:

In [23]:
mean_squared_error(y_train, pred,squared=False)

759.0600752359645

In [24]:
r2_score(y_train, pred)

0.8416632435493496

In [25]:
from sklearn.ensemble import RandomForestRegressor

In [26]:
RF = RandomForestRegressor()

In [27]:
RF.fit(X_train,y_train)

RandomForestRegressor()

In [28]:
mean_squared_error(y_train, RF.predict(X_train),squared=False)

272.3434028449924

In [29]:
r2_score(y_train,RF.predict(X_train))

0.9796172318329988

In [30]:
y_pred = LR.predict(X_test)

**Question 4. d)**

The rmse and r^2 value for test using Linear Regression and Random Forest are:

In [31]:
mean_squared_error(y_test, y_pred,squared=False)

774.1783135397773

In [32]:
r2_score(y_test,y_pred)

0.8493698992205666

In [33]:
RF.fit(X_test,y_test)

RandomForestRegressor()

In [34]:
mean_squared_error(y_test, RF.predict(X_test),squared=False)

287.6877860024648

In [35]:
r2_score(y_test,RF.predict(X_test))

0.979199555709627

**Question 4. e)**

**atemp has the highest positive correlation and windspeed has the highest negative correlation, these are the features which give significant contribution.**

**The model has a good fit with r square value of 0.979 while using Random Forest and 0.849 using Linear Regression.**

**The same goes for rmse value.**

**This indicates that Random Forest has a better fit as compared to Linear Regression for this model.**

**The model fits the data well using Linear Regression which answers to the question how well the model fits.**